# Pandas - Merging, grouping, aggregation and complex relationships

In [ ]:
# Necesssary import evil

import jupy_helpers
import pandas as pd
import numpy as np
from IPython.display import display, HTML, IFrame

np.random.seed(42)

In [ ]:
%matplotlib inline  
# This will enable us to see plots embedded in the noteboo

In [ ]:
# make the cells wide
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
con = 'sqlite:///./workshop_data.sqlite'

In [ ]:
# Read the data that we stored in the previous examples
imdb_titles = pd.read_sql('imdb_titles', con)
imdb_ratings = pd.read_sql('imdb_ratings', con)
boxoffice = pd.read_sql('boxoffice', con)
rotten_tomatoes = pd.read_sql("rotten_tomatoes", con)

In [ ]:
# Repeat the operations from the previous notebook for a clean start
imdb_titles["title_type"] = imdb_titles.titleType.astype("category")
imdb_titles.loc[imdb_titles.isAdult > 1, 'isAdult'] = 0
imdb_titles = imdb_titles.dropna(subset=["startYear"])
imdb_titles["runtimeMinutes"] = pd.to_numeric(imdb_titles.runtimeMinutes, errors="coerce").astype("Int64")
movie_titles = imdb_titles[imdb_titles.titleType.isin(["movie", "tvMovie"])]
movie_titles = movie_titles.drop(columns=["titleType", "endYear"]).dropna()

## Merging data

We would like to attach ratings to the movies data set. 
Let's merge (join) the two two IMDB data sets. That's quite simple becase they share the `tconst` columns.

### Simple merging - IMDB ratings

In [ ]:
imdb_ratings.sample(10)

Luckily, the datasets have a common column (tconst) and can be merged easily:

In [ ]:
movies_rated = movie_titles.merge(imdb_ratings, on='tconst', how='inner')

In [ ]:
movies_rated.head(10)

In [ ]:
# Alternative (based on automatic index matching)
movie_titles.set_index("tconst").join(imdb_ratings.set_index("tconst")).head(10)

### Advanced merging

Then we have two more data sets to merge: `rotten_tomatoes` and `boxoffice`. 

In [ ]:
rotten_tomatoes.sample(5)

In [ ]:
boxoffice.sample(5)

Observations :(
1. No `tconst` 
2. (year) in titles

**Exercise:**: 
1. Create `year` column with year simply parsed from `Title` by slicins and convert to `int` type.
2. Create `correctedTitle` column with ` (year)` removed.

In [ ]:
%exercise

# rotten_tomatoes["year"] = rotten_tomatoes.Title.str.slice(..., ...)
# rotten_tomatoes["Title"] = rotten_tomatoes.Title.str.slice_replace(..., ..., ...)

rotten_tomatoes["year"] = pd.to_numeric(rotten_tomatoes.Title.str.slice(-5, -1))
rotten_tomatoes["correctedTitle"] = rotten_tomatoes.Title.str.slice_replace(-7, None, '')

In [ ]:
%validate

assert (rotten_tomatoes.year[:5] == [2018, 2015, 2017, 1927, 2017]).all()
assert rotten_tomatoes.iloc[2].correctedTitle == 'Wonder Woman'

So no title `endswith` `")"` any more, right?

In [ ]:
rotten_tomatoes[rotten_tomatoes.correctedTitle.str.endswith(')')].head(10)

Have you heard of regural expressions? Get yourselves trained at https://regex101.com/ :)

Shortcut for now. To match strings like `"(Shichinin no Samurai)"` at the end of a string, we can use this regex: `\(.*\)$`.

The second ingredient to use is the `.str` accessor (see https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html), which can help us to `extract` and `replace` the year from the titles.

In [ ]:
rotten_tomatoes.correctedTitle.str.extract(r'\((.*)\)$').dropna().head()

We have just extracted the original titles. Let's construct `primaryTitle` by replacing the original titles in the `Title` column by empty strings. We also need to delete any trailing white space characters. 

In [ ]:
rotten_tomatoes["primaryTitle"] = rotten_tomatoes.correctedTitle.str.replace(r'(\s*\(.*\))$', '')

See how we succeeded ;)

In [ ]:
rotten_tomatoes[rotten_tomatoes.correctedTitle.str.contains("\\(")][["Title", "correctedTitle", "primaryTitle", "year"]].sample(10)

Feeling like a regex expert already? Try to write a RFC email address validator ... or just use an existing one: http://www.ex-parrot.com/~pdw/Mail-RFC822-Address.html 😈 

What about duplicated titles?

In [ ]:
# Use the duplicated method
rotten_tomatoes[rotten_tomatoes.primaryTitle.duplicated(keep=False)].sort_values("primaryTitle").head(20)

Seems like the differences are in genres and **years**. Let's make it easy and 

**Exercise:** Create an `honest_tomatoes` data set from `rotten_tamatoes`
1. with no duplicates (use `duplicated` by `primaryTitle` and `year` columns with a different `keep` option for indexing),
2. without `Title` and `correctedTitle` columns.

In [ ]:
%exercise

# honest_tomatoes = rotten_tomatoes.loc[___].drop(___)

honest_tomatoes = (rotten_tomatoes
                   .loc[~rotten_tomatoes.duplicated(subset=["primaryTitle", "year"], keep='first')]
                   .drop(columns=["Title", "correctedTitle"])
                  )

In [ ]:
%validate

assert honest_tomatoes[honest_tomatoes.primaryTitle == "Hairspray"].year.tolist() == [1988, 2007]
assert sorted(honest_tomatoes.columns) == ['Genres', 'No. of Reviews', 'Rank', 'RatingTomatometer', 'primaryTitle', 'year']

See what are the honestly duplicated titles:

In [ ]:
honest_tomatoes[honest_tomatoes.primaryTitle.duplicated(keep=False)].sort_values("primaryTitle")

Luckily, the boxoffice data are somewhat easier to work with.

In [ ]:
boxoffice["primaryTitle"] = boxoffice.title.str.rstrip().str.replace(r'\s*\(.*\)$', '')

And now, let's do the big big merge:

In [ ]:
movies = pd.merge(
    movies_rated,
    pd.merge(honest_tomatoes, boxoffice, on=["primaryTitle", "year"], how="left"),
    left_on=["primaryTitle", "startYear"],
    right_on=["primaryTitle", "year"],
)

In [ ]:
movies.sample(10)

In [ ]:
movies.describe()

In [ ]:
unique_movies = movies.loc[~movies.duplicated(subset=["primaryTitle", "year"], keep='first')]

# And just to make sure
unique_movies = unique_movies.dropna()

In [ ]:
unique_movies.describe()

In [ ]:
unique_movies.shape[0], movies_rated.shape[0], boxoffice.shape[0], honest_tomatoes.shape[0]

We have much less titles compared to IMDB or even boxoffice data :-| We could probably improve the merge by sanitizing the titles, likely there are punctuation, white space, Roman vs. Arabic numerals nuances.

## Visual analysis

### Matplotlib

In [ ]:
import matplotlib.pyplot as plt

Is the number of votes related to rating?

In [ ]:
movies_rated.plot(kind="scatter", x="numVotes", y="averageRating")

We can use `scatter_matrix` to scatter-plot all variables against each other.

In [ ]:
fig = pd.plotting.scatter_matrix(unique_movies, figsize=(10, 7))

Hmm, maybe not every column is useful.

**Exercise:** Use only some columns for scatter plot.

In [ ]:
%exercise

# fig = pd.plotting.scatter_matrix(___, figsize=(10, 7))

fig = pd.plotting.scatter_matrix(unique_movies[
    ["numVotes", "averageRating", 'RatingTomatometer', 'No. of Reviews']
], figsize=(10, 7))

### Seaborn

[Seaborn](https://seaborn.pydata.org/) is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics.

In [ ]:
import seaborn as sns

There are convenient functions to set plot styles and colout palettes.

In [ ]:
sns.set_context("notebook")
sns.set_style("whitegrid")
sns.set_palette("hls")

A powerful tool to visualize pairwise relationships is `pairplot`.

In [ ]:
sns.pairplot(unique_movies[["startYear", "averageRating", "numVotes", "RatingTomatometer", "lifetime_gross"]])

**Exercise:** Adapt the example from https://seaborn.pydata.org/examples/pair_grid_with_kde.html to match this figure.

![pairgrid_exercise.png](pairgrid_exercise.png)

In [ ]:
%exercise

# df = sns.load_dataset("iris")

# g = sns.PairGrid(df, diag_sharey=False)
# g.map_lower(sns.kdeplot)
# g.map_upper(sns.scatterplot)
# g.map_diag(sns.kdeplot, lw=3)


g = sns.PairGrid(
    movies[["startYear", "averageRating", "numVotes", "RatingTomatometer", "lifetime_gross"]].dropna(), 
    diag_sharey=False,
)
g.map_upper(sns.kdeplot)
g.map_lower(sns.scatterplot)
g.map_diag(sns.distplot, hist=True, kde=True);

Do you see any interesting relationships?

### Adding interactivity to plots

Seaborn is great. But maybe we would like to add some interactivity: zooming, panning, ... you name it. 
Some of that we can get with just using a different [Matplotlib backend](https://matplotlib.org/3.1.0/tutorials/introductory/usage.html#backends).
There are libraries for that. We'll show [Plotly Express](https://www.plotly.express/) here, which builds Seaborn-like interface on top of https://plot.ly/.

There are more libraries you can explore when you have more time: https://bokeh.pydata.org/en/latest/, https://altair-viz.github.io/, http://holoviews.org.

In [ ]:
import plotly_express as px

In [ ]:
px.scatter(
    movies.dropna(), 
    x="averageRating", 
    y="RatingTomatometer",
    marginal_x="histogram", 
    marginal_y="histogram",
    color="numVotes", 
    size="lifetime_gross",
    hover_name="title",
)

## Grouping & aggregation

A common pattern in data analysis is grouping (or binning) data based on some property and getting some aggredate statistics.

*Example:* Group this workshop participants by nationality a get the cardinality (the size) of each group.

In [ ]:
# we use all title types here
# let's merge first
imdb_data = imdb_titles.merge(imdb_ratings, on="tconst")
imdb_data.sample(3)

In [ ]:
grouped_by_type = imdb_data.groupby('titleType')

What did we get? 

In [ ]:
grouped_by_type

What's this `DataFrameGroupBy` object? [Its use case is](http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html):
* Splitting the data into groups based on some criteria.
* Applying a function to each group independently.
* Combining the results into a data structure.


Let's try a simple aggregate: mean of rating per type group:

In [ ]:
grouped_by_type.averageRating.mean().sort_values()

Movies are worst rated, TV episodes and video games are much better rated. What does that mean?

`GroupBy` objects have `agg` method which is quite versatile to describe what aggregations we need. As an example:

In [ ]:
grouped_by_type.agg({"averageRating": ["mean", "median", "std"], "startYear": ["min", "max", "median"]})

Grouping is also behind Seaborn's `FacetGrid` plots of categorical data

In [ ]:
g = sns.FacetGrid(imdb_data, col="titleType", col_wrap=4)
# g.map(plt.hist, "averageRating", density=True)
g.map(sns.distplot, "averageRating", norm_hist=True)

What if we were to group by decade? We don't have a decade column but we can just calculate the decades and use it for `groupby`.

In [ ]:
group_by_decade = movies_rated.groupby((np.floor_divide(movies_rated.startYear, 10) * 10))

**Exercise:** Use `group_by_decade.agg` similarly to above to get the mean of average rating and the total number of votes in each decade.

In [ ]:
%exercise

# decade_statistics = group_by_decade.agg({___})

decade_statistics = group_by_decade.agg({"averageRating": "mean", "numVotes": "sum"})

# Display (do not edit)
decade_statistics["averageRating"].plot(kind="bar")
decade_statistics


In [ ]:
%validate

assert np.allclose(decade_statistics.iloc[-1].averageRating, 6.364, rtol=0.01)
assert np.allclose(decade_statistics.iloc[-1].numVotes, 2.55e+08, rtol=0.01)

70's, 80's, 90's are the dark ages of the film industry?

**Exercise:**: Find the most profitable film for each studio. Use `groupby` and either `apply` with `pn.nlargest` or `sort_values` and `first`.

In [ ]:
%exercise

# result = movies.groupby ...

result = movies.groupby("studio").apply(pd.DataFrame.nlargest, n=1, columns="lifetime_gross").nlargest(10, columns="lifetime_gross")[["title", "startYear", "lifetime_gross"]]

# display 
result.nlargest(10, columns="lifetime_gross")[["title", "startYear", "lifetime_gross"]]

In [ ]:
%validate

assert result.loc["Sony", "lifetime_gross"].values == 373585825

## Pivoting

> pivot (third-person singular simple present pivots, present participle pivoting, simple past and past participle pivoted)
 **To turn on an exact spot.**
 
> A pivot table is a table of statistics that summarizes the data of a more extensive table ...
> Although pivot table is a generic term, Microsoft Corporation trademarked PivotTable in the United States in 1994.

Our pivoting task: Get a table with numbers of titles per year (as row) and type (as column).

One approach is to use `groupby`, `count` aggregation and `unstack`.

In [ ]:
grouped_by_year_and_type = imdb_data.groupby(['startYear', 'titleType'])

In [ ]:
pivoted = (
    grouped_by_year_and_type
    .numVotes
    .count()
    .unstack()
)
pivoted.tail()

There's a shortcut though, see if you we can use it.

**Exercise:** Create the `pivoted` table using `pivot_table`:

In [ ]:
%exercise

# pivot_table = imdb_data.pivot_table(values=___, index=___, columns=___, aggfunc=___)

pivot_table = imdb_data.pivot_table(values="numVotes", index="startYear", columns="titleType", aggfunc="count")

# display - do not edit
pivot_table.tail()

In [ ]:
%validate

pd.testing.assert_frame_equal(pivoted, pivot_table)

We can now use this to plot a kind of a histogram with colour for title types.

In [ ]:
with sns.color_palette("Paired"):
    fig, ax = plt.subplots(figsize=(16, 6))
    pivoted.loc[1990:].plot.bar(stacked=True, ax=ax)

## Final mini-project - creative, unbounded, free-style

Here are some ideas of what you can do with the data.

* Create 5-star rating based on quantiles using `quantile` and `cut` or `qcut`.
* Group by studio / decade / rating
* Compare simple (arithmetic) mean `averageRating` in each group with `averageRating` average weighted by `numVotes` ($ \frac{\sum \rm{averageRating} \times \rm{numVotes}} {\sum \rm{numVotes}} $). Use `apply` and the `wavg` function from https://pbpython.com/weighted-average.html. This function is quite time and memory consuming and thus not ideal for large data sets. You can try to implement weighted average using standard `mean`. Check the performance with the `%timeit` magic.
* Use the 5-star rating for `hue` in an interesting seaborn plot (see https://seaborn.pydata.org/tutorial/relational.html)
* Use `sns.catplot` to visualize the distrubution of incomes in each 5-star rating group. 

A couple more ideas can be found in https://github.com/brandon-rhodes/pycon-pandas-tutorial

After you have solved all of those, come up with your own quests - we may still be around and help you :-D

**TODO: remove the rest of the notebook!!!**

In [ ]:
# df = imdb_data.copy()
df = unique_movies.copy()

In [ ]:
sorted(df.columns)

### Star rating

In [ ]:
star_rating = pd.qcut(df.averageRating, 5, labels=np.arange(1, 6))

In [ ]:
df['star_rating'] = star_rating.astype(int)

In [ ]:
grouped_by_stars = df.groupby("star_rating")

In [ ]:
grouped_by_stars.agg({"lifetime_gross": ["sum", "mean"]})

In [ ]:
sns.catplot(x="star_rating", y="lifetime_gross", kind="box", dodge=False, data=df, aspect=2)

### Weighted average

In [ ]:
# https://pbpython.com/weighted-average.html

# This solution works but is very slow

def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()
    

# grouped_avg1 = grouped.apply(wavg, "averageRating", "numVotes")

In [ ]:
# Weighted average using standard functions

df["votes_rating"] = df["numVotes"] * df["averageRating"]

pd.DataFrame(
    grouped_by_stars.votes_rating.sum().div(grouped_by_stars.numVotes.sum()), columns=['weighted_mean']
).assign(simple_mean=grouped_by_stars.averageRating.mean())

In [ ]:
movies_rated["votes_rating"] = movies_rated["numVotes"] * movies_rated["averageRating"]

bins = list(range(1900, 2021, 10))
movies_rated["decade"] = pd.cut(movies_rated.startYear, bins=bins, labels=bins[:-1])

movies_by_decade = movies_rated.groupby("decade")

decade_stats = pd.DataFrame(
    movies_by_decade.votes_rating.sum().div(movies_by_decade.numVotes.sum()), columns=['weighted_mean']
).assign(simple_mean=movies_by_decade.averageRating.mean())
decade_stats.plot()

In [ ]:
fig = sns.catplot(x="decade", y="averageRating", kind="boxen", dodge=False, data=movies_rated, aspect=2)
decade_stats.plot(ax=fig.ax)

### 10 most profitable studios plotted

In [ ]:
boxoffice.groupby('studio').lifetime_gross.sum().nlargest(10).plot(kind="bar")

In [ ]:
largest_studios = unique_movies.groupby('studio').lifetime_gross.sum().nlargest(10).index

### Plots

In [ ]:
sns.catplot(
    x="studio", 
#     y="averageRating", 
    y="lifetime_gross",
    kind="box", 
    dodge=False, 
    data=unique_movies[unique_movies.studio.isin(largest_studios)], 
    aspect=2,
)

In [ ]:
px.scatter(
    movies.dropna(), 
    x="averageRating", 
    y="lifetime_gross",
    marginal_x="histogram", 
    marginal_y="histogram",
#     color="numVotes", 
    size="numVotes",
    hover_name="title",
#     log_y=True,
    trendline='ols',
)